In [1]:
# Import Libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC

In [7]:
spam_dataframe = pd.read_csv("spam_ham_dataset.csv")
spam_dataframe.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [8]:
spam_dataframe.drop("Unnamed: 0", axis=1, inplace=True)
spam_dataframe.head()

,label,text,label_num
0,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,spam,"Subject: photoshop , windows , office . cheap ...",1
4,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [9]:
spam_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5171 entries, 0 to 5170
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   label      5171 non-null   object
 1   text       5171 non-null   object
 2   label_num  5171 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 121.3+ KB


In [10]:
# We can covert "label" columns with pd.get_dummies to get 0 and 1
# but we already got a necessary column
# spam_dataframe["label"] = pd.get_dummies(spam_dataframe["label"], dtype='int', drop_first=True)

In [12]:
label_encoder = LabelEncoder()
spam_dataframe["text"] = label_encoder.fit_transform(spam_dataframe["text"])

In [ ]:
spam_dataframe.head()

,label,text,label_num
0,ham,1209,0
1,ham,1985,0
2,ham,2774,0
3,spam,3158,1
4,ham,3613,0
